# Библиотеки

In [ ]:
import torch
from tqdm import tqdm
import torchvision
import random
import numpy as np
from IPython.display import clear_output

try:
    import transformers
except ModuleNotFoundError:
    !pip install transformers
    !pip install transformers sentencepiece --quiet
    clear_output()
    import transformers
from transformers import (
    T5ForConditionalGeneration, T5TokenizerFast, T5Tokenizer,
    DataCollatorForLanguageModeling, TrainingArguments, Trainer,
)

try:
    import datasets
except ModuleNotFoundError:
    !pip install datasets
    clear_output()
from datasets import load_dataset

try:
    import rouge
except ModuleNotFoundError:
    ! pip install rouge
    clear_output()
from rouge import Rouge 

Фиксируем сиды

In [ ]:
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)
torch.use_deterministic_algorithms(True)

Пути для сохранения (не забудьте подключить диск):

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks'
saved_model_path = path + '/archive'
checkpoint_path = path + '/t5-model-small'
dataset_path = path + '/dataset'
model_name = "cointegrated/rut5-small"

Устройство ускорителя:

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Main

Функции:

In [ ]:
def make_dataset_(data, tokenizer, max_length_text=2890, max_length_ref=200):
    '''

    '''
    dataset = []
    for inst in data:
        txt = tokenizer(inst['text'], add_special_tokens=True, max_length=max_length_text, padding="max_length", truncation=True)
        sum_ = tokenizer(inst['summary'], add_special_tokens=True, max_length=max_length_ref, padding="max_length", truncation=True).input_ids
        txt["labels"] = sum_
        dataset.append(txt)
    return dataset

def save(path, model, optimizer):
    '''

    '''
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, path)

def tests_res(data, model, tokenizer, max_length_text=2890):
    '''

    '''
    res, ref = [], []
    for inst in data:
        inp = tokenizer(inst['text'], add_special_tokens=True, max_length=max_length_text, padding="max_length", truncation=True, return_tensors='pt').to(device) 
        gen_ref = tokenizer.decode(model.generate(input_ids=inp.input_ids, attention_mask=input.attention_mask)[0], skip_special_tokens=True)
        res.append(gen_ref)
        ref.append(inst['summary'])
    return res, ref

def summarize(text, model, tokenizer, max_length_text=2890):
    '''

    '''
    inp = tokenizer(text, add_special_tokens=True, max_length=max_length_text, padding="max_length", truncation=True, return_tensors='pt').to(device) 
    return tokenizer.decode(model.generate(input_ids=inp.input_ids, attention_mask=input.attention_mask)[0], skip_special_tokens=True)

def make_dataset(model, tokenizer, max_length_text=2890, max_length_ref=200, n=50):
    '''

    '''
    dataset = []
    for i in range(n):
        with open(dataset_path + f'/{i}.txt') as f:
            data = f.read()
        data = data.split('\n\n')
        txt = tokenizer(data[1], add_special_tokens=True, max_length=max_length_text, padding="max_length", truncation=True)
        sum_ = tokenizer(data[2], add_special_tokens=True, max_length=max_length_ref, padding="max_length", truncation=True).input_ids
        txt["labels"] = sum_
        dataset.append(txt)
    return dataset

Загрузка модели:

In [ ]:
# Загрузка модели
tokenizer = T5Tokenizer.from_pretrained(model_name)
model_t5 = T5ForConditionalGeneration.from_pretrained(model_name)
optimizer = torch.optim.AdamW(model_t5.parameters(),lr=1e-4)

checkpoint = torch.load(saved_model_path + '/model_t5_small_4.pth', map_location='cpu')
model_t5.load_state_dict(checkpoint['model_state_dict'])
model_t5.to(device)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

Загрузка датасета Gazeta и создание обучающей + валидационной выборок

In [ ]:
# Загрузка датасета
dataset_train = load_dataset('IlyaGusev/gazeta', revision="v2.0")["train"]
dataset_val = load_dataset('IlyaGusev/gazeta', revision="v2.0")["validation"]
dataset_test = load_dataset('IlyaGusev/gazeta', revision="v2.0")["test"]
clear_output()

In [ ]:
# Создание обучающей и валидационной выборок
print('Making train dataset')
dataset_train = make_dataset_(dataset_train, tokenizer)
print('Making val dataset')
dataset_val = make_dataset_(dataset_val, tokenizer)
print('Done!')

## Обучение языковой модели на задачу реферирования:

Если возникла ошибка:

In [ ]:
#if "Using the `Trainer` with `PyTorch` requires `accelerate`: Run `pip install --upgrade accelerate`" occured:
! pip uninstall -y transformers accelerate
! pip install transformers accelerate
# then reload

In [ ]:
# параметры для Тренера
training_args = TrainingArguments(
    output_dir= checkpoint_path,
    overwrite_output_dir=True,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    warmup_steps=10,
    gradient_accumulation_steps=16,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    seed=42,
)

# Тренер
trainer = Trainer(
    model=model_t5,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    tokenizer=tokenizer,
    optimizers = (optimizer, None)
)

In [ ]:
# Обучение модели
model_t5.train()
logs = trainer.train()
print('Saving model')
save(saved_model_path + '/model_t5_small_5.pth', model_t5, optimizer)

## Тестирование:

Тест на тестовой выборке датасета Gazeta

In [ ]:
# Тестирование модели
model_t5.test()
print('Testing')
model_results, refs = tests_res(dataset_test, model_t5, tokenizer)
print('Done! Counting Rouge')
rouge = Rouge()
scores = rouge.get_scores(model_results, refs, avg=True)
print(scores)

Тест на более коротких текстах (тезисы к ВКР 2020)

In [ ]:
for i in range(64):
    t = open(path + f'/texts/text_{i}', 'r', encoding='cp1251').read()
    print(f'{i}) {summarize(t)}')
    print('-----------')

## Дообучение на своем датасете:

Загрузка датасета и создание выборки 

In [ ]:
train_set = make_dataset(model_t5, tokenizer)

Обучение:

In [ ]:
model_t5.train()

# параметры для Тренера
training_args = TrainingArguments(
    output_dir= checkpoint_path,
    overwrite_output_dir=True,
    per_device_train_batch_size=2,
    num_train_epochs=1,
    warmup_steps=10,
    gradient_accumulation_steps=16,
    save_strategy="epoch",
    load_best_model_at_end=True,
    seed=42,
)

# Тренер
trainer = Trainer(
    model=model_t5,
    args=training_args,
    train_dataset=train_set,
    tokenizer=tokenizer,
    optimizers = (optimizer, None)
)

## Тестирование:

Тест на более коротких текстах (тезисы к ВКР 2020)

In [ ]:
model_t5.eval()

In [ ]:
for i in range(64):
    t = open(path + f'/texts/text_{i}', 'r', encoding='cp1251').read()
    print(f'{i}) {summarize(t)}')
    print('-----------')